## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Qaanaaq,GL,77.4840,-69.3632,-7.82,95,100,7.25,overcast clouds
1,1,Atuona,PF,-9.8000,-139.0333,77.74,75,9,17.11,clear sky
2,2,Saint-Philippe,RE,-21.3585,55.7679,83.79,84,64,9.17,broken clouds
3,3,Mataura,NZ,-46.1927,168.8643,50.49,99,27,2.80,scattered clouds
4,4,Cidreira,BR,-30.1811,-50.2056,74.66,57,0,13.65,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_temperature_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]
preferred_temperature_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Atuona,PF,-9.8000,-139.0333,77.74,75,9,17.11,clear sky
2,2,Saint-Philippe,RE,-21.3585,55.7679,83.79,84,64,9.17,broken clouds
10,10,Mayumba,GA,-3.4320,10.6554,82.15,78,64,10.74,broken clouds
11,11,Rikitea,PF,-23.1203,-134.9692,77.67,76,100,11.05,light rain
19,19,Sibu,MY,2.3000,111.8167,77.31,94,40,0.00,heavy intensity rain
20,20,Kruisfontein,ZA,-34.0033,24.7314,75.88,42,0,14.79,clear sky
25,25,Banda Aceh,ID,5.5577,95.3222,80.01,81,75,10.78,broken clouds
27,27,Bani,DO,18.2833,-70.3333,75.85,73,11,6.60,few clouds
28,28,Vaini,TO,-21.2000,-175.2000,75.36,100,75,9.22,light rain
29,29,Guanambi,BR,-14.2233,-42.7814,81.19,63,100,2.46,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_temperature_df.count()

City_ID                200
City                   200
Country                194
Lat                    200
Lng                    200
Max Temp               200
Humidity               200
Cloudiness             200
Wind Speed             200
Current Description    200
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_temperature_df = preferred_temperature_df.dropna(how='any',axis=0)
preferred_temperature_df.count()

City_ID                194
City                   194
Country                194
Lat                    194
Lng                    194
Max Temp               194
Humidity               194
Cloudiness             194
Wind Speed             194
Current Description    194
dtype: int64

In [13]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_temperature_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
hotel_df = hotel_df.round({'Lat': 2, 'Lng':2})
# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Atuona,PF,77.74,clear sky,-9.80,-139.03,
2,Saint-Philippe,RE,83.79,broken clouds,-21.36,55.77,
10,Mayumba,GA,82.15,broken clouds,-3.43,10.66,
11,Rikitea,PF,77.67,light rain,-23.12,-134.97,
19,Sibu,MY,77.31,heavy intensity rain,2.30,111.82,
...,...,...,...,...,...,...,...
653,Gao,ML,87.15,clear sky,16.64,1.64,
656,Anjangaon,IN,79.12,clear sky,21.16,77.31,
657,Bandarbeyla,SO,77.97,clear sky,9.49,50.81,
659,Vila Velha,BR,82.94,overcast clouds,-20.33,-40.29,


In [14]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # Add the latitude and longitude location key for the params dictionary
    params["Location"]= f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url,params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,"Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError): 
        print("Hotel not found ... skipping.")
        

        

Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not 

In [16]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.tail(50)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
506,Kieta,PG,80.10,moderate rain,-6.22,155.63,
512,Port-Gentil,GA,87.76,scattered clouds,-0.72,8.78,
513,Quatre Cocos,MU,84.09,few clouds,-20.21,57.76,
517,Cockburn Town,TC,76.86,clear sky,21.46,-71.14,
519,Adrar,MR,81.05,few clouds,20.50,-10.07,
521,Fuerte Olimpo,PY,82.63,clear sky,-21.04,-57.87,
522,Vao,NC,80.06,light rain,-22.67,167.48,
523,Barentu,ER,86.18,broken clouds,15.11,37.59,
525,Port Blair,IN,80.64,mist,11.67,92.75,
527,Tiarei,PF,78.82,broken clouds,-17.53,-149.33,


In [10]:
# 8a. Create the output File (CSV)

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

NameError: name 'clean_hotel_df' is not defined

In [ ]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [ ]:
# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure
